In [3]:
def fcfs(processos):
    processos = sorted(processos, key=lambda x: x.arrival_time)
    tempo_atual = 0

    for process in processos:
        if tempo_atual < process.arrival_time:
            tempo_atual = process.arrival_time
        process.tempo_espera = tempo_atual - process.arrival_time
        process.turnaround_time = process.tempo_espera + process.burst_time
        tempo_atual += process.burst_time

    avg_tempo_espera = sum([p.tempo_espera for p in processos]) / len(processos)
    avg_turnaround_time = sum([p.turnaround_time for p in processos]) / len(processos)

    print(f"{'Processo':<10} {'Tempo de Espera':<15} {'Tempo de Execução':<15}")
    for p in processos:
        print(f"{p.name:<10} {p.tempo_espera:<15} {p.turnaround_time:<15}")

    print(f"\nTempo de Espera Médio: {avg_tempo_espera:}")
    print(f"Tempo de Execução Médio: {avg_turnaround_time:}")


In [5]:
def round_robin(processos, quantum):
    time = 0
    fila_prontos = []
    processos_completos = 0
    n = len(processos)

    while processos_completos < n:
        # Adicione processos que chegaram ao tempo atual na fila de pronto
        for p in processos:
            if p.arrival_time == time and p.tempo_faltante > 0:
                fila_prontos.append(p)

        # Execute o processo na frente da fila de pronto
        if fila_prontos:
            processo_atual = fila_prontos.pop(0)

            if processo_atual.tempo_faltante > quantum:
                time += quantum
                processo_atual.tempo_faltante -= quantum

                # Adicione novamente à fila se não estiver concluído
                for p in processos:
                    if p.arrival_time <= time and p.tempo_faltante > 0 and p not in fila_prontos:
                        fila_prontos.append(p)

                if processo_atual.tempo_faltante > 0:
                    fila_prontos.append(processo_atual)
            else:
                time += processo_atual.tempo_faltante
                processo_atual.turnaround_time = time - processo_atual.arrival_time
                processo_atual.tempo_espera = processo_atual.turnaround_time - processo_atual.burst_time
                processo_atual.tempo_faltante = 0
                processos_completos += 1
        else:
            time += 1

    avg_tempo_espera = sum([p.tempo_espera for p in processos]) / len(processos)
    avg_turnaround_time = sum([p.turnaround_time for p in processos]) / len(processos)

    print(f"{'Processo':<10} {'Tempo de Espera':<15} {'Tempo de Execução':<15}")
    for p in processos:
        print(f"{p.name:<10} {p.tempo_espera:<15} {p.turnaround_time:<15}")

    print(f"\nTempo de Espera Médio: {avg_tempo_espera:}")
    print(f"Tempo de Execução Médio: {avg_turnaround_time:}")


In [6]:
def sjf(processos):
    n = len(processos)
    completo = 0
    tempo_atual = 0
    processos = sorted(processos, key=lambda x: (x.arrival_time, x.burst_time))

    while completo < n:
        processos_disponiveis = [p for p in processos if p.arrival_time <= tempo_atual and p.turnaround_time == 0]

        # Se não houver processos disponíveis, apenas avance o tempo
        if not processos_disponiveis:
            tempo_atual += 1
            continue

        # Escolha o processo com o menor burst time
        next_process = min(processos_disponiveis, key=lambda x: x.burst_time)
        tempo_atual += next_process.burst_time
        next_process.turnaround_time = tempo_atual - next_process.arrival_time
        next_process.tempo_espera = next_process.turnaround_time - next_process.burst_time
        completo += 1

    avg_tempo_espera = sum([p.tempo_espera for p in processos]) / n
    avg_turnaround_time = sum([p.turnaround_time for p in processos]) / n

    print(f"{'Processo':<10} {'Tempo de Espera':<15} {'Tempo de Execução':<15}")
    for p in processos:
        print(f"{p.name:<10} {p.tempo_espera:<15} {p.turnaround_time:<15}")

    print(f"\nTempo de Espera Médio: {avg_tempo_espera:}")
    print(f"Tempo de Execução Médio: {avg_turnaround_time:}")


In [7]:
def srtf(processos):
    tempo_atual = 0
    processos_completos = 0
    n = len(processos)
    processo_atual = None

    while processos_completos < n:
        # Encontrar os processos que já chegaram
        processos_disponiveis = [p for p in processos if p.arrival_time <= tempo_atual and not p.completo]

        # Se não houver processos disponíveis, avance o tempo
        if not processos_disponiveis:
            tempo_atual += 1
            continue

        # Selecione o processo com o menor tempo restante
        shortest_process = min(processos_disponiveis, key=lambda x: x.tempo_faltante)

        # Se não há processo atual ou se o processo atual tem um tempo restante maior que o novo processo
        if not processo_atual or processo_atual.tempo_faltante > shortest_process.tempo_faltante:
            processo_atual = shortest_process

        # Decrementa o tempo restante do processo atual
        processo_atual.tempo_faltante -= 1
        tempo_atual += 1

        # Verificar se o processo atual foi concluído
        if processo_atual.tempo_faltante == 0:
            processo_atual.completo = True
            processo_atual.turnaround_time = tempo_atual - processo_atual.arrival_time
            processo_atual.tempo_espera = processo_atual.turnaround_time - processo_atual.burst_time
            processos_completos += 1
            processo_atual = None

    avg_tempo_espera = sum([p.tempo_espera for p in processos]) / n
    avg_turnaround_time = sum([p.turnaround_time for p in processos]) / n

    print(f"{'Processo':<10} {'Tempo de Espera':<15} {'Tempo de Execução':<15}")
    for p in processos:
        print(f"{p.name:<10} {p.tempo_espera:<15} {p.turnaround_time:<15}")

    print(f"\nTempo de Espera Médio: {avg_tempo_espera:}")
    print(f"Tempo de Execução Médio: {avg_turnaround_time:}")


In [8]:
def priority_non_preemptive(processos):
    tempo_atual = 0
    processos_completos = 0
    n = len(processos)
    processos = sorted(processos, key=lambda x: x.arrival_time)  # Ordenar por tempo de chegada inicialmente

    while processos_completos < n:
        # Encontrar os processos que já chegaram
        processos_disponiveis = [p for p in processos if p.arrival_time <= tempo_atual and p.turnaround_time == 0]

        # Se não houver processos disponíveis, avance o tempo
        if not processos_disponiveis:
            tempo_atual += 1
            continue

        # Selecione o processo com a prioridade mais alta (valor mais baixo indica alta prioridade)
        processo_maior_prioridade = min(processos_disponiveis, key=lambda x: x.priority)
        tempo_atual += processo_maior_prioridade.burst_time
        processo_maior_prioridade.turnaround_time = tempo_atual - processo_maior_prioridade.arrival_time
        processo_maior_prioridade.tempo_espera = processo_maior_prioridade.turnaround_time - processo_maior_prioridade.burst_time
        processos_completos += 1

    avg_tempo_espera = sum([p.tempo_espera for p in processos]) / n
    avg_turnaround_time = sum([p.turnaround_time for p in processos]) / n

    print(f"{'Processo':<10} {'Prioridade':<10} {'Tempo de Espera':<15} {'Tempo de Execução':<15}")
    for p in processos:
        print(f"{p.name:<10} {p.priority:<10} {p.tempo_espera:<15} {p.turnaround_time:<15}")

    print(f"\nTempo de Espera Médio: {avg_tempo_espera:}")
    print(f"Tempo de Execução Médio: {avg_turnaround_time:}")


In [9]:
def priority_preemptive(processos):
    tempo_atual = 0
    processos_completos = 0
    n = len(processos)
    processo_atual = None

    while processos_completos < n:
        # Encontrar os processos que já chegaram
        processos_disponiveis = [p for p in processos if p.arrival_time <= tempo_atual and not p.completo]

        # Se não houver processos disponíveis, avance o tempo
        if not processos_disponiveis:
            tempo_atual += 1
            continue

        # Selecione o processo com a prioridade mais alta (valor mais baixo indica alta prioridade)
        processo_maior_prioridade = min(processos_disponiveis, key=lambda x: (x.priority, x.arrival_time))

        # Se há um processo em execução e a sua prioridade é menor que o processo atualmente em execução
        if processo_atual and processo_maior_prioridade.priority < processo_atual.priority:
            processo_atual = processo_maior_prioridade

        # Se não há processo em execução, selecione o processo de maior prioridade
        if not processo_atual:
            processo_atual = processo_maior_prioridade

        # Se o processo atual não foi executado antes ou foi interrompido
        if processo_atual.ultima_execucao_em != tempo_atual - 1:
            processo_atual.tempo_espera += tempo_atual - max(processo_atual.ultima_execucao_em, processo_atual.arrival_time)

        processo_atual.ultima_execucao_em = tempo_atual
        processo_atual.tempo_faltante -= 1
        tempo_atual += 1

        # Verificar se o processo atual foi concluído
        if processo_atual.tempo_faltante == 0:
            processo_atual.completo = True
            processo_atual.turnaround_time = tempo_atual - processo_atual.arrival_time
            processos_completos += 1
            processo_atual = None

    avg_tempo_espera = sum([p.tempo_espera for p in processos]) / n
    avg_turnaround_time = sum([p.turnaround_time for p in processos]) / n

    print(f"{'Processo':<10} {'Prioridade':<10} {'Tempo de Espera':<15} {'Tempo de Execução':<15}")
    for p in processos:
        print(f"{p.name:<10} {p.priority:<10} {p.tempo_espera:<15} {p.turnaround_time:<15}")

    print(f"\nTempo de Espera Médio: {avg_tempo_espera:}")
    print(f"Tempo de Execução Médio: {avg_turnaround_time:}")


In [10]:
class Process:
    def __init__(self, name, arrival_time, burst_time, priority=None):
        self.name = name
        self.arrival_time = arrival_time
        self.burst_time = burst_time
        self.tempo_faltante = burst_time
        self.priority = priority
        self.tempo_espera = 0
        self.turnaround_time = 0
        self.completo = False
        self.ultima_execucao_em = -1

if __name__ == '__main__':
    print("\nCódigo realizado por: \n Leonardo Bonato Bizaro 081200014 \n Nathan Bitu de Oliveira 081200004 \n Pedro de Souza Silva 081200017")
    n = int(input("Digite o número de processos: "))
    processos = []

    for i in range(n):
        name = input(f"\nNome do processo {i+1}: ")
        arrival_time = int(input(f"Tempo de chegada do processo {name}: "))
        burst_time = int(input(f"Tempo de execução do processo {name}: "))
        priority = None
        has_priority = input(f"O processo {name} tem uma prioridade? (s/n) ").lower().strip()
        if has_priority == 's':
            priority = int(input(f"Prioridade do processo {name} (número menor indica maior prioridade): "))
        processos.append(Process(name, arrival_time, burst_time, priority))

    print("\nEscolha o algoritmo de escalonamento:")
    print("1. FCFS")
    print("2. Round Robin")
    print("3. SJF")
    print("4. SRTF")
    print("5. Prioridade Não Preemptiva")
    print("6. Prioridade Preemptiva")

    choice = int(input("\nDigite o número correspondente ao algoritmo: "))

    if choice == 1:
        fcfs(processos)
    elif choice == 2:
        quantum = int(input("\nDigite o quantum para o Round Robin: "))
        round_robin(processos, quantum)
    elif choice == 3:
        sjf(processos)
    elif choice == 4:
        srtf(processos)
    elif choice == 5:
        priority_non_preemptive(processos)
    elif choice == 6:
        priority_preemptive(processos)
    else:
        print("Escolha inválida.")


Escolha o algoritmo de escalonamento:
1. FCFS
2. Round Robin
3. SJF
4. SRTF
5. Prioridade Não Preemptiva
6. Prioridade Preemptiva
Processo   Tempo de Espera Tempo de Execução
AAA        0               4              

Tempo de Espera Médio: 0.0
Tempo de Execução Médio: 4.0
